Cell 1 – Imports

In [1]:
# Import the 'os' module to interact with the operating system (e.g., list files in a folder)
import os

# Import the EfficientNetB7 model and its preprocessing function from Keras applications
from tensorflow.keras.applications.efficientnet import EfficientNetB7, preprocess_input

# Import utility functions to load an image file and convert it to a NumPy array
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Import the Model class to create a custom model that outputs features instead of class predictions
from tensorflow.keras.models import Model


Cell 2 – Load the EfficientNet-B7 model

In [2]:
# Load the EfficientNet-B7 model with pre-trained ImageNet weights
# By default, include_top=True, so the final classification layers are included
base_model = EfficientNetB7(weights='imagenet')


268326632/268326632 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Cell 3 – Restructure the model (remove final classification layer)

In [3]:
# Create a new model that has the same input as the original EfficientNet-B7 model
# but uses the second-to-last layer's output (before the final classification layer) as the output
feature_extractor_model = Model(
    inputs=base_model.inputs,               # Use the same input tensor as EfficientNet-B7
    outputs=base_model.layers[-2].output    # Use the layer just before the final prediction layer
)


Cell 4 – Show model summary

In [4]:
# Print a summary of the modified model architecture to verify the output shapes and layers
print(feature_extractor_model.summary())


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 600, 600,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 600, 600,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 600, 600,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 600, 600,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 601, 601,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 300, 300,  │      1,728 │ stem_conv_pad[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 300, 300,  │        256 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 300, 300,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 300, 300,  │        576 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 300, 300,  │        256 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 300, 300,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 64)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 64)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 16)  │      1,040 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 64)  │      1,088 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 300, 300,  │          0 │ block1a_activati… │
│ (Multiply)          │ 64)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 300, 300,  │      2,048 │ block1a_se_excit

 Total params: 64,097,687 (244.51 MB)

 Trainable params: 63,786,960 (243.33 MB)

 Non-trainable params: 310,727 (1.19 MB)

None


Cell 5 – Prepare feature storage and image directory

In [5]:
# Create an empty dictionary to store extracted features for each image
features = {}

# Define the directory that contains the input images
# Make sure this path points to the folder where your images are stored
directory = 'image data/'


Cell 6 – Loop through images, preprocess, and extract features

In [6]:
# Loop through every file name in the specified image directory
for image_name in os.listdir(directory):
    # Build the full path to the current image file by concatenating directory and file name
    image_path = directory + image_name

    # Load the image from disk and resize it to 600x600 pixels (required size for EfficientNet-B7)
    img = load_img(image_path, target_size=(600, 600))

    # Convert the loaded PIL image to a NumPy array (height, width, channels)
    img = img_to_array(img)

    # Add a batch dimension so the array shape becomes (1, height, width, channels)
    # This matches the expected input shape for the model
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

    # Preprocess the image using EfficientNet's preprocessing (scaling, normalization, etc.)
    img = preprocess_input(img)

    # Pass the preprocessed image through the feature extractor model to get the feature vector
    feature_vector = feature_extractor_model.predict(img, verbose=0)

    # Store the extracted feature vector in the dictionary using the image path as the key
    features[image_path] = feature_vector


In [7]:
features

{'image data/00bb4d84c9cf8903eeb0671eb66bd739.png': array([[ 0.03815603, -0.10182183, -0.15579796, ..., -0.2027399 ,
         -0.2207806 , -0.1652356 ]], dtype=float32)}

In [10]:
# Print the shape of the feature vector extracted for this single image
print(feature_vector.shape)

(1, 2560)
